In [ ]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import sys
import numpy as np
import re
import random

from skimage import io
from skimage import img_as_ubyte
from skimage import filters
from google.colab.patches import cv2_imshow



In [ ]:
DIR = '/content/drive/My Drive/Colab Notebooks/Dog/dog/' #before using code check path of images

In [ ]:
dog_dir = os.listdir(DIR)

In [ ]:
def rotation(image):
  rows, columns = image.shape
  R = cv2.getRotationMatrix2D((columns/2, rows/2), random.randint(-70, 70), 1)
  return cv2.warpAffine(gray_image, R, (columns, rows))
  

In [ ]:
def flip_image(image):
  return np.fliplr(image)

In [ ]:
def image_blur(image):
  return filters.gaussian(image, sigma=1)

In [ ]:
def zoom_image(image, zoom_factor):
  # let len = 100, width = 100, zoom_factor = 1.5
  len, width = image.shape[:2]
  new_len, new_width = int(len * zoom_factor), int(width * zoom_factor)
  # new_len = 150, new_width = 150

  #finding center of image after zoom
  center_x, center_y = new_len//2, new_width//2 
  # center_x = 75, center_y = 75

  # finding coordinates of our original size picture 
  x1, y1 = (center_x - len//2) ,(center_y - width//2) 
  # x1 = 25, y1 = 25 
  x2, y2 = (center_x + len//2), (center_y + width//2)
  # x2 = 125, y2 = 125
  # y2 - y1 = 100 = len
  # x2 - x1 = 100 = width
   
  box = np.array([x1, y1, x2, y2])
  # scaling coordinates to our original picture
  box = (box / zoom_factor).astype(np.int) 
  # box = [16 16 83 83]
  # now rezing cropped image to len x width i.e. (100 x 100)
  return cv2.resize(image[y1:y2, x1:x2],(len, width))

  


In [ ]:
def shrink_image(image, shrink_factor):

  len, width = image.shape[:2]
  result_image = np.full((len, width),0, dtype=np.uint8) # empty image with original image size

  new_len, new_width = int(len * shrink_factor), int(width * shrink_factor)
  img = cv2.resize(image, (new_len, new_width )) # shrink original image
  
  center_x, center_y = len//2, width//2 
  x1, y1 = (center_x - new_len//2), (center_y - new_width//2)
  x2, y2 = (center_x + new_len//2), (center_y + new_width//2)
  # copying original shrinked image in original size image
  if x2-x1 == new_width and y2-y1 == new_len:
    result_image[y1:y2, x1:x2] = img
  else:
    result_image[y1:y2+1, x1:x2+1] = img
  return result_image



In [ ]:
count = 0
for i, name in enumerate(dog_dir):
  #nm = re.split('\.',name)[0]
  label = re.split('_', name)[1] + re.split('_', name)[2]
  img = cv2.imread(DIR + name)
  img = cv2.resize(img,(100, 100), interpolation=cv2.INTER_CUBIC)# firstly resizing picture
  
  gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # changing picture into grayscale
  
  newDirPath = "/content/drive/My Drive/Colab Notebooks/Dog/new_dog/"# directory to save
 
  if not os.path.exists(newDirPath):# if dir doesn't exist then create
    os.makedirs(newDirPath)
 
  # original image
  io.imsave(newDirPath+str(count)+str(random.randint(1,1000))+label, gray_image)# original
  count+=1
  io.imsave(newDirPath+str(count)+str(random.randint(1,100))+label, shrink_image(gray_image,  random.choice(np.linspace(0.5,0.9,20)))) # random shrink
  count+=1
  io.imsave(newDirPath+str(count)+str(random.randint(400,600))+label, zoom_image(gray_image,random.choice(np.linspace(1.3,1.5,20)))) # random zoom
  count+=1
  # 5 rotated images, rotated+zoom, rotated+shrink , total 15 each cycle
  for i in range(5):
    rotated_image_8bit = img_as_ubyte(rotation(gray_image)) 
    io.imsave(newDirPath+str(count)+str(random.randint(1,100))+label, cv2.cvtColor(rotated_image_8bit, cv2.COLOR_GRAY2BGR))# random rotate
    count += 1
    io.imsave(newDirPath+str(count)+str(random.randint(50,300))+label, shrink_image(rotated_image_8bit,  random.choice(np.linspace(0.5,0.9,20)))) # random shrink
    count+=1
    io.imsave(newDirPath+str(count)+str(random.randint(400,600))+label, zoom_image(rotated_image_8bit,random.choice(np.linspace(1.3,1.5,20)))) # random zoom
    count+=1
  # flipped image
  flipped = img_as_ubyte(flip_image(gray_image))
  io.imsave(newDirPath+str(count)+str(random.randint(1,100))+label, flipped)
  count+=1
  io.imsave(newDirPath+str(count)+str(random.randint(400,600))+label, zoom_image(flipped,random.choice(np.linspace(1.3,1.5,20)))) #  flip + random zoom
  count+=1
  # blurred image
  blurred = img_as_ubyte(image_blur(gray_image))
  io.imsave(newDirPath+str(count)+str(random.randint(1,100))+label, blurred)
  count+=1
  io.imsave(newDirPath+str(count)+str(random.randint(400,600))+label, zoom_image(blurred,random.choice(np.linspace(1.3,1.5,20)))) # blur + random zoom
  count+=1
  